### Importing necessary libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math
import time
from datetime import datetime
from sqlalchemy import create_engine, text

<div align = 'center'>
<h1>Games Data Scrap</h>
</div>

### Defining necessary variables

In [2]:
rows_by_page = 200 ## You can choose between 50, 100 or 200

## MODE FULL | DELTA, It will determine if it's a full or an incremental load of the data.
try:
    MODE = PMODE ## If It comes from another notebook
except:
    MODE = "DELTA"
## Full load: We will scrap every existing record on webpage
## Incremental load: We will scrap the latest 1000 updated records. 
print(f"Mode: {MODE}")

# URL of vgchartz table, on its first page. Everything ordered by "Last Update Date"
# To get the latest data.
url = f"https://www.vgchartz.com/games/games.php?page=1&results={str(rows_by_page)}&order=LastUpdate&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1"

# Creating an empty list to save each webpage table
data = []

## To identify from which page we need to retry
retry_page = 0

Mode: FULL


### Necessary Functions

In [3]:
def text_to_date(text):
    if pd.isnull(text):
        return text
    else:
        # Discard "th" from the text
        non_suffix_date = text.replace('th', '').replace('st', '').replace('nd', '').replace('rd', '')
        date = datetime.strptime(non_suffix_date, '%d %b %y').date()
        return date


def web_scrapping (number_of_pages, rows_by_page, initial_page):
    try: ##We won't validate response code, because even when it fails, Its status code == 200. Try/Except will work better
        for x in range(initial_page, number_of_pages+1): ## If it fails at any page, we can easily replace the "1" by the last succesfully scrapped page
            time.sleep(5) ## To avoid timeout error
            url = f"https://www.vgchartz.com/games/games.php?page={str(x)}&results={str(rows_by_page)}&order=LastUpdate&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1"

            response = requests.get(url)

            soup = BeautifulSoup(response.text, 'html.parser')

            counter = 0 ## We need to add a column on the third row (because of webpage's table format)
            table = soup.find('div', {'id': 'generalBody'}).find('table')
            every_row = table.find_all('tr')

            if len(every_row) <= 3:
                print(f"No results returned on page {x} :(")
                raise ValueError(f"No results returned on page {x} :(")

            # Iterating over every row
            for row in every_row:

                # Getting cells from every row
                cells = row.find_all(['th', 'td'])

                row_data = [] ## A list with every table's row
                for cell in cells:
                    img = cell.find('img')

                    if img:
                        img = img.replace_with(img.text)
                        img = str(img).split('"')[1] #We need "alt" attr value for getting the "Console" value
                        row_data.append(str(img))

                    else: ##It's not an image, so we just strip the data string value
                        row_data.append(cell.text.strip())

                if (counter == 2):
                    row_data.insert(2, 'GameName')

                global data

                data.append(row_data) ## Saving current row into "every row" list

                counter = counter + 1 ## Number of rows iterated

            print(f"Page {str(x)} Succesfully Scrapped!")
    except Exception as e:
        global retry_page
        if retry_page != 0:
            raise ValueError(f'Page {str(x)} Request Failed. With the following message: {str(e)}')
        else:
            retry_page = x
            print(f"Failed on page {str(x)}. Retrying....")

### Web Scrapping

In [4]:
# Making a request on first page, to get the number of Results.
response = requests.get(url) 

# Code 200 means everything is ok!
if response.status_code == 200:
    # parsing HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Finding table inside webpage
    table = soup.find('div', {'id': 'generalBody'}).find('table')
    
    if MODE == "FULL":
        ## Getting total number of games, to realize how many times we should iterate over the url page number
        first_row = table.find('tr')
        results = first_row.find('th')
        results = str(results.text).replace('Results: (', '').replace(')', '').replace(',', '')
        results = int(results)

        number_of_pages = math.ceil(results/rows_by_page)
    else: ## If we making an incremental refresh, we will iterate over the first 1000 latest updated videogames.
        results = 1000
        number_of_pages = 5
    print("Number of videogames: ", results, " - ", "Number of Pages: ", number_of_pages)

    web_scrapping (number_of_pages, rows_by_page, 1)
    ## If It fails, we will retry the web scrapping from that page
    if retry_page != 0:
        time.sleep(5)
        web_scrapping (number_of_pages, rows_by_page, retry_page)

    videogames_sales_df = pd.DataFrame(data)

else:
    raise ValueError(f'Request on first page Failed. Status Code: {response.status_code}')


Number of videogames:  63916  -  Number of Pages:  320
Page 1 Succesfully Scrapped!
Page 2 Succesfully Scrapped!
Page 3 Succesfully Scrapped!
Page 4 Succesfully Scrapped!
Page 5 Succesfully Scrapped!
Page 6 Succesfully Scrapped!
Page 7 Succesfully Scrapped!
Page 8 Succesfully Scrapped!
Page 9 Succesfully Scrapped!
Page 10 Succesfully Scrapped!
Page 11 Succesfully Scrapped!
Page 12 Succesfully Scrapped!
Page 13 Succesfully Scrapped!
Page 14 Succesfully Scrapped!
Page 15 Succesfully Scrapped!
Page 16 Succesfully Scrapped!
Page 17 Succesfully Scrapped!
Page 18 Succesfully Scrapped!
Page 19 Succesfully Scrapped!
Page 20 Succesfully Scrapped!
Page 21 Succesfully Scrapped!
Page 22 Succesfully Scrapped!
Page 23 Succesfully Scrapped!
Page 24 Succesfully Scrapped!
Page 25 Succesfully Scrapped!
Page 26 Succesfully Scrapped!
Page 27 Succesfully Scrapped!
Page 28 Succesfully Scrapped!
Page 29 Succesfully Scrapped!
Page 30 Succesfully Scrapped!
Page 31 Succesfully Scrapped!
Page 32 Succesfully Scra

In [5]:
videogames_sales_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,"Results: (63,916)",<< 1 2 3 4 5 6 >>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,A B C D E F G H I J...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Pos,Game,GameName,Console,Publisher,Developer,VGChartz Score,Critic Score,User Score,Total Shipped,Total Sales,NA Sales,PAL Sales,Japan Sales,Other Sales,Release Date,Last Update
3,1,Boxart Missing,Star Wars: The Force Unleashed - Ultimate Sith...,PS3,Unknown,LucasArts,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,06th Jan 24
4,2,Boxart Missing,Star Wars: The Force Unleashed - Ultimate Sith...,X360,Unknown,LucasArts,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,06th Jan 24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64871,63912,Boxart Missing,[Prototype],X360,Activision,Radical Entertainment,N/A,7.9,N/A,N/A,1.31m,0.84m,0.35m,N/A,0.12m,09th Jun 09,N/A
64872,63913,Boxart Missing,[Prototype] Read the review,PC,Activision,Radical Entertainment,8.1,7.9,N/A,N/A,N/A,N/A,N/A,N/A,N/A,09th Jun 09,N/A
64873,63914,Boxart Missing,[Prototype],PSN,Activision,Radical Entertainment,N/A,7.8,N/A,N/A,N/A,N/A,N/A,N/A,N/A,14th Dec 10,N/A
64874,63915,Boxart Missing,_summer ##,PS2,GN Software,GN Software,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,24th Aug 06,N/A


In [6]:
videogames_sales_df.head(7)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,"Results: (63,916)",<< 1 2 3 4 5 6 >>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,A B C D E F G H I J...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Pos,Game,GameName,Console,Publisher,Developer,VGChartz Score,Critic Score,User Score,Total Shipped,Total Sales,NA Sales,PAL Sales,Japan Sales,Other Sales,Release Date,Last Update
3,1,Boxart Missing,Star Wars: The Force Unleashed - Ultimate Sith...,PS3,Unknown,LucasArts,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,06th Jan 24
4,2,Boxart Missing,Star Wars: The Force Unleashed - Ultimate Sith...,X360,Unknown,LucasArts,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,06th Jan 24
5,3,Boxart Missing,Star Wars: The Force Unleashed - Ultimate Sith...,PC,Unknown,LucasArts,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,06th Jan 24
6,4,Boxart Missing,Contra,PC,Unknown,Konami,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,05th Jan 24


### Format Adaptations

In [7]:
videogames_sales_df = videogames_sales_df[( videogames_sales_df[videogames_sales_df.columns[0]].str.isnumeric() ) | ( videogames_sales_df[videogames_sales_df.columns[0]] == 'Pos' ) ]  ## We only need these rows to discard each page header
videogames_sales_df.columns = videogames_sales_df.iloc[0] ## First Row as header
videogames_sales_df = videogames_sales_df[1:] ##Discard First row (now header)
videogames_sales_df = videogames_sales_df[videogames_sales_df["Pos"].str.isnumeric()] ## We've already gotten the headers, so we don't need these rows 

In [8]:
videogames_sales_df.head(10)

2,Pos,Game,GameName,Console,Publisher,Developer,VGChartz Score,Critic Score,User Score,Total Shipped,Total Sales,NA Sales,PAL Sales,Japan Sales,Other Sales,Release Date,Last Update
3,1,Boxart Missing,Star Wars: The Force Unleashed - Ultimate Sith...,PS3,Unknown,LucasArts,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,06th Jan 24
4,2,Boxart Missing,Star Wars: The Force Unleashed - Ultimate Sith...,X360,Unknown,LucasArts,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,06th Jan 24
5,3,Boxart Missing,Star Wars: The Force Unleashed - Ultimate Sith...,PC,Unknown,LucasArts,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,06th Jan 24
6,4,Boxart Missing,Contra,PC,Unknown,Konami,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,05th Jan 24
7,5,Boxart Missing,Contra,NS,Unknown,Konami,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,05th Jan 24
8,6,Boxart Missing,Contra,MSX,Unknown,Konami,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,05th Jan 24
9,7,Boxart Missing,Contra,Mob,Unknown,Konami,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,05th Jan 24
10,8,Boxart Missing,Contra,C64,Unknown,Konami,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,05th Jan 24
11,9,Boxart Missing,Pac-Man,PC,Unknown,Namco,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,05th Jan 24
12,10,Boxart Missing,Pac-Man,MSX,Unknown,Namco,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,05th Jan 24


### Filters and needed transformations

In [9]:
videogames_sales_df.columns

Index(['Pos', 'Game', 'GameName', 'Console', 'Publisher', 'Developer',
       'VGChartz Score', 'Critic Score', 'User Score', 'Total Shipped',
       'Total Sales', 'NA Sales', 'PAL Sales', 'Japan Sales', 'Other Sales',
       'Release Date', 'Last Update'],
      dtype='object', name=2)

In [10]:
videogames_sales_df.drop(columns= {"Game", "Pos"}, inplace=True) ##To discard each page "Boxart Missing" message, and position in the results
videogames_sales_df= videogames_sales_df.replace('N/A', pd.NA)

In [11]:
videogames_sales_df.head(10)

2,GameName,Console,Publisher,Developer,VGChartz Score,Critic Score,User Score,Total Shipped,Total Sales,NA Sales,PAL Sales,Japan Sales,Other Sales,Release Date,Last Update
3,Star Wars: The Force Unleashed - Ultimate Sith...,PS3,Unknown,LucasArts,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,06th Jan 24
4,Star Wars: The Force Unleashed - Ultimate Sith...,X360,Unknown,LucasArts,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,06th Jan 24
5,Star Wars: The Force Unleashed - Ultimate Sith...,PC,Unknown,LucasArts,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,06th Jan 24
6,Contra,PC,Unknown,Konami,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,05th Jan 24
7,Contra,NS,Unknown,Konami,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,05th Jan 24
8,Contra,MSX,Unknown,Konami,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,05th Jan 24
9,Contra,Mob,Unknown,Konami,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,05th Jan 24
10,Contra,C64,Unknown,Konami,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,05th Jan 24
11,Pac-Man,PC,Unknown,Namco,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,05th Jan 24
12,Pac-Man,MSX,Unknown,Namco,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,05th Jan 24


### Date columns Transformations

In [12]:
videogames_sales_df["Release Date"] = videogames_sales_df["Release Date"].apply(text_to_date)
videogames_sales_df["Last Update"] = videogames_sales_df["Last Update"].apply(text_to_date)
videogames_sales_df[["Release Date", "Last Update"]] = videogames_sales_df[["Release Date", "Last Update"]].apply(pd.to_datetime, errors='coerce')

### Numeric columns Transformations

In [13]:
videogames_sales_df[['VGChartz Score', 'Critic Score', 'User Score']] = videogames_sales_df[['VGChartz Score', 'Critic Score', 'User Score']].\
                                                          apply(pd.to_numeric, errors='coerce', downcast='float')

In [14]:
videogames_sales_df[['Total Shipped', 'Total Sales', 'NA Sales', 'PAL Sales', 'Japan Sales', 'Other Sales']] = \
videogames_sales_df[['Total Shipped', 'Total Sales', 'NA Sales', 'PAL Sales', 'Japan Sales', 'Other Sales']].apply(lambda x : x.str.replace('m', '')).\
                                                                                            apply(pd.to_numeric, errors='coerce', downcast='float').\
                                                                                            apply(lambda x : x * 1000000.0)

In [15]:
videogames_sales_df.sample(10)

2,GameName,Console,Publisher,Developer,VGChartz Score,Critic Score,User Score,Total Shipped,Total Sales,NA Sales,PAL Sales,Japan Sales,Other Sales,Release Date,Last Update
32874,Futurama,PS2,VU Games,Unique Development Studios Sweden,NaN,NaN,NaN,NaN,70000.0,40000.0,30000.0,NaN,10000.0,2003-08-14,NaT
42716,Mistborn: Birthright,PS3,Little Orbit,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-12-31,NaT
50483,Road Runner,NES,Tengen,Tengen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1989-01-01,NaT
58350,The Incredible Hulk,MS,U.S. Gold,Probe Entertainment Limited,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01,NaT
29956,E.O.S. - Exhibition of Speed,DC,Titus,Player 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001-08-21,NaT
37220,Jak II,PS2,Sony Computer Entertainment,Naughty Dog,NaN,NaN,NaN,1600000.0,NaN,NaN,NaN,NaN,NaN,2003-10-14,NaT
36661,Impossible Mission-II,NES,SEI,Novatrade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1990-01-01,NaT
29707,Duke Assault,PC,Unknown,WizardWorks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
19344,Alex Ferguson's Player Manager 2003,PC,Ubisoft,Anco Software,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003-01-01,NaT
24902,Chaos Chronicles,PC,bitComposer Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-04-01,NaT


### String Columns Transformations

In [16]:
videogames_sales_df[['GameName', 'Console', 'Publisher', 'Developer']] = videogames_sales_df[['GameName', 'Console', 'Publisher', 'Developer']].\
                                                        apply(lambda x : x.str.upper().str.strip()).\
                                                        replace('UNKNOWN', pd.NA)

### Columns names cleaning

In [17]:
for col in videogames_sales_df.columns:
    print(col.strip().replace(' ', '_'))
    videogames_sales_df.rename(columns={col : col.strip().replace(' ', '_')}, inplace=True)

GameName
Console
Publisher
Developer
VGChartz_Score
Critic_Score
User_Score
Total_Shipped
Total_Sales
NA_Sales
PAL_Sales
Japan_Sales
Other_Sales
Release_Date
Last_Update


### Final  Results show

In [18]:
videogames_sales_df.sample(20)

2,GameName,Console,Publisher,Developer,VGChartz_Score,Critic_Score,User_Score,Total_Shipped,Total_Sales,NA_Sales,PAL_Sales,Japan_Sales,Other_Sales,Release_Date,Last_Update
56627,SURF'S UP,X360,UBISOFT,UBISOFT MONTREAL,NaN,6.2,NaN,NaN,100000.0,9.000000e+04,0.0,NaN,10000.0,2007-05-30,NaT
52417,SHARK TALE,GBA,ACTIVISION,VICARIOUS VISIONS,NaN,NaN,NaN,NaN,680000.0,4.900000e+05,180000.0,NaN,10000.0,2004-09-27,NaT
37218,JAK AND DAXTER: THE PRECURSOR LEGACY,PS2,SONY COMPUTER ENTERTAINMENT,NAUGHTY DOG,NaN,9.0,8.9,NaN,3640000.0,2.080000e+06,1090000.0,150000.0,330000.0,2001-12-04,NaT
18427,5 SPOTS PARTY,WW,COSMONAUT GAMES,COSMONAUT GAMES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-07-20,NaT
31090,FAIRY TAIL: PORTABLE GUILD,PSP,KONAMI,KONAMI,NaN,NaN,NaN,NaN,110000.0,NaN,NaN,110000.0,NaN,2010-06-03,NaT
52814,SHOGI,PSN,SONY COMPUTER ENTERTAINMENT,SONY COMPUTER ENTERTAINMENT AMERICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-04-22,NaT
21147,BABY PALS,DS,CRAVE ENTERTAINMENT,BRAIN TOYS,NaN,NaN,NaN,NaN,280000.0,2.600000e+05,10000.0,NaN,20000.0,2007-11-19,NaT
47143,PETZ VET,GBA,UBISOFT,UBISOFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-03-26,NaT
30174,EIJUKUGO TARGET 1000 DS,DS,IE INSTITUTE,IE INSTITUTE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-08-09,NaT
56817,SYNDICATE,PC,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [19]:
videogames_sales_df_count = len(videogames_sales_df.index)
print("Total Count:", videogames_sales_df_count) # Count
if videogames_sales_df_count != results:
    raise ValueError(f"{results} games found in webpage, {videogames_sales_df_count} games scrapped. Difference: {results - videogames_sales_df_count}")
print("Max Updated Date:", videogames_sales_df["Last_Update"].max())
print("Max VGZ Score (it shouldn't be > 10)", videogames_sales_df["VGChartz_Score"].max())
print("Max Shipped units:", videogames_sales_df["Total_Shipped"].max())
print(videogames_sales_df.dtypes)

Total Count: 63916
Max Updated Date: 2024-01-06 00:00:00
Max VGZ Score (it shouldn't be > 10) 10.0
Max Shipped units: 520000000.0
2
GameName                  object
Console                   object
Publisher                 object
Developer                 object
VGChartz_Score           float32
Critic_Score             float32
User_Score               float32
Total_Shipped            float32
Total_Sales              float32
NA_Sales                 float32
PAL_Sales                float32
Japan_Sales              float32
Other_Sales              float32
Release_Date      datetime64[ns]
Last_Update       datetime64[ns]
dtype: object


### Saving Individual dimensions

In [20]:
dim_videogames_df = videogames_sales_df.rename(columns={"GameName" : 'Game_Name'})["Game_Name"].drop_duplicates()
dim_videogames_df_count = len(dim_videogames_df.index)

dim_console_df = videogames_sales_df.rename(columns={"Console": "Console_Abbreviation"})["Console_Abbreviation"].drop_duplicates()
dim_console_df_count = len(dim_console_df.index)

dim_publisher_df = videogames_sales_df[videogames_sales_df["Publisher"].notnull()]["Publisher"].drop_duplicates()
dim_publisher_df_count = len(dim_publisher_df.index)

dim_developer_df = videogames_sales_df[videogames_sales_df["Developer"].notnull()]["Developer"].drop_duplicates()
dim_developer_df_count = len(dim_developer_df.index)

## Saving Final results directly to our database

### We are using our SRC schema, which will save raw data before getting dimensions ids

#### Defining our connection string, shared by every table in the process

In [21]:
table_schema = 'SRC' ## Raw Data before replacing descriptions by ids
user_name = 'XXXXX' ## We won't save credentials on github! 
password =  'XXXXX' ## We won't save credentials on github!
dw_server = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
db_name = 'VideoGames_Sales'

connection_string = (
    f'mssql+pyodbc://{user_name}:{password}@{dw_server}/'
    f'{db_name}?driver=ODBC+Driver+17+for+SQL+Server'
)

table_names = ['Fact_VideoGames_Sales', 'Dim_VideoGames', 'Dim_Console', 'Dim_Publisher', 'Dim_Developer']

### Loading every table on db

In [22]:
# Connecting with SQL Server
engine = create_engine(url=connection_string, fast_executemany=True)

for table_name in table_names:

    try:
        start_time = time.time()

        ## Before inserting new records, we must truncate the intermediate table
        ## To avoid duplicated data
        with engine.connect() as conn:
            conn.execute(text(f"TRUNCATE TABLE {table_schema}.{table_name}"))
            conn.commit() ## We must commit our changes, otherwise they won't have effect
        print(f"{table_name} Successfully truncated! :D")

        ## Inserting new data
        if table_name == 'Fact_VideoGames_Sales':
            videogames_sales_df.to_sql(name=table_name,schema=table_schema, con=engine, index=False, if_exists='append')
            df_count = videogames_sales_df_count
            print(f"Source File rows: {videogames_sales_df_count}")
        elif table_name == 'Dim_VideoGames':
            dim_videogames_df.to_sql(name=table_name,schema=table_schema, con=engine, index=False, if_exists='append')
            df_count = dim_videogames_df_count
            print(f"Source File rows: {dim_videogames_df_count}")
        elif table_name == 'Dim_Console':
            dim_console_df.to_sql(name=table_name,schema=table_schema, con=engine, index=False, if_exists='append')
            df_count = dim_console_df_count
            print(f"Source File rows: {dim_console_df_count}")
        elif table_name == 'Dim_Publisher':
            dim_publisher_df.to_sql(name=table_name,schema=table_schema, con=engine, index=False, if_exists='append')
            df_count = dim_publisher_df_count
            print(f"Source File rows: {dim_publisher_df_count}")
        elif table_name == 'Dim_Developer':
            dim_developer_df.to_sql(name=table_name,schema=table_schema, con=engine, index=False, if_exists='append')
            df_count = dim_developer_df_count
            print(f"Source File rows: {dim_developer_df_count}")

        ## Cheking and comparing inserted rows
        with engine.connect() as conn:
            rows_affected = conn.execute(text(f"select count(1) from {table_schema}.{table_name}")).scalar()

        ### Printing and comparing row numbers
        print(f"Inserted rows: {rows_affected}")

        if (df_count != rows_affected):
            raise ValueError(f"Error on {table_name}! There is a difference of {df_count - rows_affected} between source and destination")
        
        end_time = time.time()
        elapsed_time = (end_time - start_time) / 60 ##Minutes
        
        print(f"{table_name} DB Insertion Finished in {elapsed_time} minutes ! :D")
        print("#############")
    except Exception as e:
        ## Properly closing connections
        engine.dispose()
        raise ValueError(f"Job Failed with the following message! :( : {str(e)}")

## Properly closing connections
engine.dispose()
print("Finished!")

Fact_VideoGames_Sales Successfully truncated! :D
Source File rows: 63916
Inserted rows: 63916
Fact_VideoGames_Sales DB Insertion Finished in 0.06008257865905762 minutes ! :D
#############
Dim_VideoGames Successfully truncated! :D
Source File rows: 41052
Inserted rows: 41052
Dim_VideoGames DB Insertion Finished in 0.008723243077596029 minutes ! :D
#############
Dim_Console Successfully truncated! :D
Source File rows: 81
Inserted rows: 81
Dim_Console DB Insertion Finished in 0.0001665194829305013 minutes ! :D
#############
Dim_Publisher Successfully truncated! :D
Source File rows: 3381
Inserted rows: 3381
Dim_Publisher DB Insertion Finished in 0.0008462786674499512 minutes ! :D
#############
Dim_Developer Successfully truncated! :D
Source File rows: 8846
Inserted rows: 8846
Dim_Developer DB Insertion Finished in 0.0018828352292378744 minutes ! :D
#############
Finished!
